In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import _tree
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X_train = pd.read_pickle('./data/Race_x_train.pkl')
y_train = pd.read_pickle('./data/Race_y_train_so.pkl')
X_test = pd.read_pickle('./data/Race_x_test.pkl')
y_test = pd.read_pickle('./data/Race_y_test_so.pkl')

In [ ]:
model= RandomForestClassifier(random_state = 100)
model.fit(X_train, y_train)

# Measure edge importance

In [ ]:
# NxN-es feature mátrixot tölti fel
def fill_mx(matrix, feature_list, test_list):
    l = len(test_list)
    for i in range(l-1):
        for j in range(i+1,l):
            x = feature_list.index(test_list[i])
            y = feature_list.index(test_list[j])
            if (x==y): continue
            matrix[min(x,y)][max(x,y)] += 1

In [ ]:
def recurse(node, list):
    if tree_.feature[node] != _tree.TREE_UNDEFINED:
        list.append(feature_name[node])
        recurse(tree_.children_left[node], list)
        recurse(tree_.children_right[node], list)
        del list[-1]
    else:
        fill_mx(matrix, feature_names, list)
                
feature_names = X_train.columns.values.tolist()
feature_list = []
matrix = np.array(np.zeros((len(feature_names),len(feature_names))))

#start recurse
for tree in model.estimators_:
    tree_ = tree.tree_
    feature_name = [
            feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
            for i in tree_.feature
        ]
    recurse(0, feature_list)

print(matrix.shape)
print(matrix)

In [ ]:
df = pd.DataFrame(matrix, index=feature_names, columns=feature_names)
pairs = df.stack().sort_values(ascending=False)
pairs[0:20]

In [ ]:
# sum edge weights for each node

n = 128
top = np.zeros(n)

for i in range(n):
    sum = np.sum(np.hstack((matrix[0:i,i] , matrix[i,i:n])))
    top[i] = sum

In [ ]:
order = np.sort(top)[::-1]
order_i = np.argsort(top)[::-1]

for v, i in zip(order,order_i):
    print("f{} \t {}".format(i+1,v))

In [ ]:
import matplotlib.pyplot as plt 

# edge counting
strings = []
for i in order_i[0:15]:
    strings.append("f{}".format(str(i+1)))

# sklearn gini
ms_imp_df = pd.DataFrame({'feature': X_train.columns.values,
                       'importance': model.feature_importances_})
 
# Reorder by importance
ms_ordered_df = ms_imp_df.sort_values(by='importance', ascending=False)
data = ms_ordered_df.head(n=15)

# plot
plt.figure(figsize=(20,8))

plt.subplot(121)
plt.barh(strings,order[0:15])
plt.title("edge weight counting")
plt.gca().invert_yaxis()

plt.subplot(122)
plt.title("matplotlib feature_importance (gini index)")
plt.barh(data["feature"],data["importance"])
plt.gca().invert_yaxis()

# Remove top x features 

In [ ]:
# original model with 128 features
accuracy_score(y_test, model.predict(X_test))

In [ ]:
results = []
for n in range(0,101,5):
    print("removing: {} features".format(n))
    strings = []
    for i in order_i[0:n]:
        strings.append("f{}".format(str(i+1)))

    # remove features
    X_train_drop = X_train.drop(columns=strings)
    X_test_drop = X_test.drop(columns=strings)

    # retrain model
    model_drop= RandomForestClassifier(random_state = 100)
    model_drop.fit(X_train_drop, y_train)

    # new model accuracy score
    results.append((n,accuracy_score(y_test, model_drop.predict(X_test_drop))))

In [ ]:
res = [[ i for i, j in results ],
       [ j for i, j in results ]]

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(res[0], res[1])
plt.ylim(0.7)
plt.title("Race prediction model - top features removed")
plt.ylabel("accuracy")
plt.xlabel("features removed")
plt.show()